# Part 1 - Building a Language Model

#### Francesca Maria Mizzi - 118201L
#### B.Sc in IT (Honours) (Artificial Intelligence)
#### ICS2203 - Natural Language Processing: Methods and Tools

In the following Jupyter Notebook, I will be creating 3 different types on N-gram models based on the (Baby) British National Corpus; vanilla language model, laplace language model and UNK language model. The n-grams I will use are unigrams, bigrams and trigrams. 

The first step in order to create the language model is importing all the relevant modules. As you can see, I mostly made use of NLTK in order to manipulate the corpus however I also used sklearn in order to split my dataset to train and test as well as numpy, random, mpmath and sys in order to generate the probabilities, perplexity and the sentence generation

In [1]:
from nltk.corpus.reader.bnc import BNCCorpusReader
from nltk.collocations import *
from collections import Counter
from sklearn.model_selection import train_test_split
from nltk.lm import MLE
from nltk.util import ngrams
from nltk.corpus import stopwords
import nltk, re, pprint, string
from nltk import word_tokenize, sent_tokenize
import numpy
import random
import mpmath as mp
import sys
from datetime import datetime
import os
import psutil

## Extraction and Pre-Processing

As can be seen below, I made use of the NLTK module BNCCorpusReader. This module doesn't do too much other than reading the files specified in the fileids variable and parsing the text within the xml file.

Due to the way the module is initialized, it is very easy to include more files from within the (Baby) British National Corpus simply by adding them to the list fileids.

I did not submit the whole (Baby) British National Corpus but only the files which I used below. Should you wish to add more files from the corpus, they need to be downloaded and put in the appropriate location as shown below.

In [2]:
bnc_reader = BNCCorpusReader(root="BNC/Texts", fileids=r'[A-K]/\w*/\w*\.xml')
fileids = ['aca/A6U.xml', 'aca/ACJ.xml']

Before building the language models, the corpus must first be pre-processed. Pre-processing allows us to filter out any symbols or phrases which are not desired within the corpus.

As can be seen, I use the BNCCorpusReader in order to extract the raw sentences from the corpus. I then loop through each sentence word by word and check if there are any symbols within the word. If symbols are present, that symbol is removed and the new word appended to the list of tokens. If there are no symbols within the word, the word is automatically added to the list of tokens.

It is also to be noted that before looping through the words of a sentence, the start of sentence token is added to indicate the beginning of a new sentence. When it has looped through an entire sentence, it appends the end of sentence token to the list in order to indicate the end of a sentence. These tokens are especially important when generating the sentences later on. 

After removing all the symbols, all the characters are made lowercase in order to facilitate word prediction and sentence generation later on since the language model does not consider a word with a lowercase character and an uppercase character to be the same. For example, to the language model, "the" is not the same as "The" since the first letter is uppercase.

In [3]:
raw_sents = BNCCorpusReader.sents(bnc_reader, fileids=fileids)
punct = "“”‘’!\"#$€%&()*'+-,./:;<=>?@[\]^_`{|}~\n"
temp = []
        
for sentence in raw_sents:
    temp.append("<s>")
    for word in sentence:
        for letter in word:
            if letter in punct:
                word.replace(letter,'')
        temp.append(word)
    temp.append("</s>")
    
tokens = [x.lower() for x in temp]

I then split the corpus tokens into test and train sets using the module from sklearn. In my case, I chose to make the test set 20% of the total corpus but that can easily be changed by adjusting the second variable.

In [4]:
train_words, test_words = train_test_split(tokens, test_size = 0.2)

## Language Models

Now begins the building of the language models. I build 3 different types of models: vanilla (a basic language model), laplace (+1 smoothing added to the vanilla model) and UNK model (replacing tokens with a frequency less than 2 by UNK tokens and using the laplace model).

### Vanilla Model

In order to make my code as efficient as possible, I tried to code as modular as possible in order to easily call specific methods.

For the vanilla model, I created 3 different functions, unigram, bigram and trigram models.

For the unigram model, I use the counter module to count each instance of each word within the training set. Then, for every word within the new model, I take the instance count and divide it by the total number of words within the training set.

For the bigram model, I first found all the bigrams within the corpus by taking each word, finding the word before it and adding the tuple to a list of bigrams. The same process as the unigram model is performed but this time dividing instead with the instance count for the first word (unigram) of the bigram.

The trigram model is very similar to the bigram model where the bigrams and trigrams are found by find the word before each word and using the count of each trigran and dividing it by the bigram count of the first two words.

In [19]:
def vanilla_uni(train_words):
    unigram = Counter(train_words)
    
    for word in unigram:
        unigram[word] = unigram[word]/len(train_words)
        
    return unigram
def vanilla_bi(train_words):
    bigram = Counter([(word, train_words[i + 1]) for i, word in enumerate(train_words[:-1])])
    counter = Counter(train_words)
    
    for word in bigram:
        bigram[word] = bigram[word]/counter[word[0]]
        
    return bigram
    
def vanilla_tri(train_words):
    bigram = Counter([(word, train_words[i + 1]) for i, word in enumerate(train_words[:-1])])
    trigram = Counter([(word, train_words[i + 1], train_words[i + 2]) for i, word in enumerate(train_words[:-2])])
    
    for word in trigram:
        trigram[word] = trigram[word]/bigram[(word[0], word[1])]
        
    return trigram

### Laplace Model

The laplace models are almost identical to the vanilla models, the only difference being that before dividing, 1 is added to the numerator in order to perform +1 smoothing. Smoothing is performed in order to generalize better. 

As done prior, 3 different methods were created for each of the n-grams.

In [6]:
def laplace_uni(train_words):
    unigram = Counter(train_words)
    
    for word in unigram:
        unigram[word] = unigram[word]+1/len(train_words)
        
    return unigram
def laplace_bi(train_words):
    bigram = Counter([(word, train_words[i + 1]) for i, word in enumerate(train_words[:-1])])
    counter = Counter(train_words)
    
    for word in bigram:
        bigram[word] = bigram[word]+1/counter[word[0]]
        
    return bigram
    
def laplace_tri(train_words):
    bigram = Counter([(word, train_words[i + 1]) for i, word in enumerate(train_words[:-1])])
    trigram = Counter([(word, train_words[i + 1], train_words[i + 2]) for i, word in enumerate(train_words[:-2])])
    
    for word in trigram:
        trigram[word] = trigram[word]+1/bigram[(word[0], word[1])]
        
    return trigram

### UNK Model

The UNK language models are based very heavily on the laplace models mentioned prior. How the UNK model works is as follows:
 - Getting the frequency counts for each word in the corpus
 - If the frequency count for that word is less than or equal to 2, that word is "removed" from the corpus and replaced by  the UNK token
 - The laplace is model is then applied to this new training set
 
 In the case of the bigram and trigram UNK models, the unigram UNK model is first created in order to check and replace each individual word and then passes it through the laplace bigram or trigram models

In [7]:
def unk_uni(train_words):
    
    counter = Counter(train_words)
    model = {}
    model["<UNK>"] = 0
    
    for word in counter:
        if counter[word] <= 2:
            model["<UNK>"] += 1
            
        else:
            model[word] = counter[word]
            
    for word in model:
        model[word] = model[word]+1/len(train_words)
        
    return model

def unk_bi(train_words):
    
    unigram = unk_uni(train_words)
    
    for i, word in enumerate(train_words):
        if not (word in unigram):
            train_words[i] = "<UNK>"
            
    return laplace_bi(train_words)

def unk_tri(train_words):
    
    unigram = unk_uni(train_words)
    
    for i, word in enumerate(train_words):
        if not (word in unigram):
            train_words[i] = "<UNK>"
            
    return laplace_tri(train_words)

## Probability

In order to carry out linear interpolation and calculate the probability of a sentence existing, we must first figure out the probability of the existence of each unigram, bigram and trigram within the sentence. In order to do this, I created 3 methods which calculate the probability of a specific n-gram. 

The probability of a unigram is called by calling the specific entry of that unigram from the specified language model.

The probability of a bigram is called by splitting up the bigram into 2 words and searching for that bigram within the language model.

The probability of a trigram is done like the probability of a bigram but instead splitting the n-gram into 3 words.

In [8]:
def uni_prob(model,unigram):
    probability = model[unigram]
    return probability

def bi_prob(model,bigram):
    first = bigram.split()[0]
    second = bigram.split()[1]
    probability = model[first,second]
    return probability

def tri_prob(model, trigram):
    first = trigram.split()[0]
    second = trigram.split()[1]
    third = trigram.split()[2]
    probability = model[first,second,third]
    return probability

In order to find the probabilty of a sentence existing using our language models, I carried out linear interpolation on the sentence. This is done by taking the probability of the unigram, bigrams and trigrams, multiplying them by their individual lambdas and adding them together. 

However, it is very clear that in most sentences, multiple unigrams, bigrams and trigrams exist. Therefore, to find the probability of each of the different types on n-grams, I used the chain rule of probability. This chain rule implies that multiplying the probabilities of each unigram gives the total unigram probability, multiplying the probabilites of each bigram gives the total bigram probability, etc. etc.

The calculation of the probability of a sentence existing is as follows:
 - The sentence is padded with the start and end tokens in order to accuratly find the frequency counts within the corpus
 - The sentence is split up into words and added to a list
 - The individual lambdas for the linear interpolation are declared
 - For the unigram probability of the sentence, the individual probability of each word in the sentence is found and appended to a list of probabilities
 - For the bigram probability of the sentence, all of the bigrams within the sentence are found. The probability of each bigram is then found and appeneded to a list of probabilities
 - For the trigram probability of the sentence, all of the trigrams within the sentence are found. The probability of each trigram is then found and appended to a list of probabilities
 
The above steps are performed on each different type of language model, depending on that is specified.

Finally, in order to carry out the linear interpolation, the total unigram, bigram and trigram probabilities are calculated by multiplying the values within each list, respectively. We take these final 3 probabilities, multiply them by their respective lambdas and add them together. The result is the probability of a sentence existing.

In [20]:
def probability (sentence, model):
    sent = "<s> "+ sentence + " </s>"
    words = sent.split()
    uni_lambda = 0.1
    bi_lambda = 0.3
    tri_lambda = 0.6
    
    unigrams_probability = []
    bigrams_probability = []
    trigrams_probability = []
    
    if model == "Vanilla":
        # unigram
                
        for word in words:
            unigrams_probability.append(uni_prob(vanilla_uni(train_words),word))
        
        # bigram
        
        bigrams = nltk.ngrams(words, 2)
        for pair in bigrams:
            bigram = ' '.join(pair)
            bigrams_probability.append(bi_prob(vanilla_bi(train_words), vanilla_uni(train_words), bigram))
        
        # trigram
        trigrams = nltk.ngrams(words, 3)
        for trio in trigrams:
            trigram = ' '.join(trio)
            trigrams_probability.append(tri_prob(vanilla_tri(train_words),vanilla_bi(train_words),trigram))
        
    elif model == "Laplace":
        # unigram
                
        for word in words:
            unigrams_probability.append(uni_prob(laplace_uni(train_words),word))
        
        # bigram
        
        bigrams = nltk.ngrams(words, 2)
        for pair in bigrams:
            bigram = ' '.join(pair)
            bigrams_probability.append(bi_prob(laplace_bi(train_words), laplace_uni(train_words), bigram))
        
        # trigram
        trigrams = nltk.ngrams(words, 3)
        for trio in trigrams:
            trigram = ' '.join(trio)
            trigrams_probability.append(tri_prob(laplace_tri(train_words),laplace_bi(train_words),trigram))
        
    elif model == "UNK":
        # unigram
                
        for word in words:
            unigrams_probability.append(uni_prob(unk_uni(train_words),word))
        
        
        # bigram
        
        bigrams = nltk.ngrams(words, 2)
        for pair in bigrams:
            bigram = ' '.join(pair)
            bigrams_probability.append(bi_prob(unk_bi(train_words), unk_uni(train_words), bigram))
        
        # trigram
        trigrams = nltk.ngrams(words, 3)
        for trio in trigrams:
            trigram = ' '.join(trio)
            trigrams_probability.append(tri_prob(unk_tri(train_words),unk_bi(train_words),trigram))
        
    prob1 = numpy.prod(unigrams_probability)
    prob2 = numpy.prod(bigrams_probability)
    prob3 = numpy.prod(trigrams_probability) 
    
    probability = (uni_lambda*prob1)+(bi_lambda*prob2)+(tri_lambda*prob3)
    return probability

## Perplexity

In order to calculate the perplexity of the sentence, I made use of the mpmath unit. This unit was used in order to hold the perplexity in a true float.

In order to calculate the perplexity, I took each line from my test set and stored the length of each line in a variable N. I then combined each line to form a sentence. I then checked if the sentence exits in my model. If it does, I multiply the current perplexity p with the inverse of the frequence of that line. If the sentence does not exist in the mdoel, the perplexity is multiplied by a very large number (sys.maxsize). The final perplexity is calculated by calculating the current perplexity p to the power of the inverse of the number of lines.

I am aware that I did not do this part correctly as I split up the test and train corpus by word and not by sentence so the perplexity is not accurate. 

In [21]:
def perplexity(test_words, model):
    
    p = mp.mpf(1)
    
    N = mp.mpf(0)
    
    for line in test_words:
        N += len(line)
        line = ' '.join(line)
        
        if model[line] > 0:
            p = p * (1/model[line])
        else:
            p = p * sys.maxsize
            
    p = pow(p, 1/float(N))
    return p

In [22]:
perplexity(test_words, vanilla_bi(train_words))

mpf('15694.328977677425')

## Generation

In order to generate the next word of a sentence, I created three methods which generates the next word based on the 3 different type on n-gram models, unigram, bigram and trigram. 

The unigram generation is done as follows:
- Creating a numpy array with the values of all the unigrams in the respective model
- Finding the average of this weight
- Choosing a word semi-randomly and getting the index and weight of this random word
- Adding this new word to the sentence
This is done until the new word generated is the stop token.

The bigram generation is done very similarly to the unigram generation with the only difference being that it takes the last two words of a sentence rather than only 1.

Unfortunately, I did not manage to make my trigram generation work. For some reason, it chooses to keep the original sentence and does not generate any new words. 


My original plan for the bigram and trigram generation was to find the probability of each bigram and trigram based on the last 1 or 2 words of the sentence, respectively, and generate the next word based on the bigram or trigram which has the highest probability. However, I quickly discovered that this took far too long to run and choose to instead take a simpler approach. 

In [23]:
def uni_generate(model, sentence, last = "", count = None):
    
    if(count != 0 and sentence[-1] != last):
        
        weights = numpy.array(list(model.values()))
        norm = weights/numpy.sum(weights)
        
        resample = numpy.random.multinomial(1, norm)
        key = list(resample).index(1)
        value = list(model.keys())[key]
        
        sentence.append(value)
        if count != None:
            uni_generate(model, sentence, last, count-1)
        else:
            uni_generate(model, sentence, last)
            
    return sentence

def bi_generate(model, sentence, last, count = None):
    if(count != 0 and sentence[-1] != last):
        
        bigrams = []
        last_word = sentence[-1]
        
        
        for entry in model:
            if entry[0] == last_word:
                bigrams.append((entry,model[entry]))
        if(bigrams == []):
            return sentence 
        
        v = [x[1] for x in bigrams]
        k = [x[0] for x in bigrams]
        weights = numpy.array(v)
        norm = weights / numpy.sum(weights)
        resample = numpy.random.multinomial(1, norm)
        key = list(resample).index(1)
        value = k[key]

        sentence.append(value[1])

        if count != None:
            bi_generate(model, sentence, last, count-1)
        else:
            bi_generate(model, sentence, last)
        
    return sentence


def tri_generate(bi_model, tri_model, sentence, last = "", count = None):
    if(len(sentence) == 1):
        sentence = BigramGenerate(bi_model, sentence, last, count=1)
        
    if(count != 0 and sentence[-1] != last):
        trigrams = []
        last_word = sentence[-1]
        
        for entry in tri_model:
            if(entry[0] == sentence[-2] and entry[1] == sentence[-1]):
                trigrams.append((entry,tri_model[entry]))
                
        if(trigrams == []):
            return sentence
        
        v = [x[1] for x in trigrams]
        k = [x[0] for x in trigrams]
        weights = np.array(v)
        norm = weights / np.sum(weights)
        resample = np.random.multinomial(1, norm)
        key = list(resample).index(1)
        value = k[key] 
        
        sentence.append(value[2])
        if count != None:
            tri_generate(bi_model, tri_model, sentence, last, count-1)
            
        else:
            tri_generate(bi_model, tri_model, sentence, last)
            
            
    return sentence

Here I created a function used to generate all 3 types of sentences based on the methods created above. This method checks to see which model is selected and calls the relevant language models.

In [24]:
def generate(model, phrase):
    if model == "1" or model == "Vanilla" or model == "vanilla":
        sent1 = ["<s>"]
        w = phrase.split()
        for word in w:
            sent1.append(word)

        sent2 = sent1.copy()
        sent3 = sent1.copy()

        print("Generating vanilla model...")
        print("GENERATED VANILLA SENTENCES")
        print("Unigram: "+ str (uni_generate(model = vanilla_uni(train_words), sentence = sent1, last = "</s>")))
        print("Bigram: "+ str (bi_generate(model = vanilla_bi(train_words), sentence = sent2, last = "</s>")))
        print("Trigram: "+ str (tri_generate(bi_model = vanilla_bi(train_words), tri_model = vanilla_tri(train_words), sentence = sent3, last = "</s>")))
        
    elif model == "2" or model == "Laplace" or model == "laplace":
        sent1 = ["<s>"]
        w = phrase.split()
        for word in w:
            sent1.append(word)

        sent2 = sent1.copy()
        sent3 = sent1.copy()

        print("Generating laplace model...")
        print("GENERATED LAPLACE SENTENCES")
        print("Unigram: "+ str (uni_generate(model = laplace_uni(train_words), sentence = sent1, last = "</s>")))
        print("Bigram: "+ str (bi_generate(model = laplace_bi(train_words), sentence = sent2, last = "</s>")))
        print("Trigram: "+ str (tri_generate(bi_model = laplace_bi(train_words), tri_model = laplace_tri(train_words), sentence = sent3, last = "</s>")))
        
    elif model == "3" or model == "UNK" or model == "unk":
        sent1 = ["<s>"]
        w = phrase.split()
        for word in w:
            sent1.append(word)

        sent2 = sent1.copy()
        sent3 = sent1.copy()

        print("Generating UNK model...")
        print("GENERATED UNK SENTENCES")
        print("Unigram: "+ str (uni_generate(model = unk_uni(train_words), sentence = sent1, last = "</s>")))
        print("Bigram: "+ str (bi_generate(model = unk_bi(train_words), sentence = sent2, last = "</s>")))
        print("Trigram: "+ str (tri_generate(bi_model = unk_bi(train_words), tri_model = unk_tri(train_words), sentence = sent3, last = "</s>")))

Below is the code used in order to generate sentences based off the users input.

In [26]:
model = input("Which model would you like to use? (1) Vanilla (2) Laplace (3) UNK : ")
phrase = input("Enter phrase to continue to generate: ")
generate(model,phrase)

Which model would you like to use? (1) Vanilla (2) Laplace (3) UNK : 1
Enter phrase to continue to generate: they all went
Generating vanilla model...
GENERATED VANILLA SENTENCES
Unigram: ['<s>', 'they', 'all', 'went', 'includes', 'dangerous', '</s>']
Bigram: ['<s>', 'they', 'all', 'went', 'of', '.', 'and', '<UNK>', 'social', 'thus', 'law', ',', 'adult', 'the', 'in', 'murder', '<s>', '<UNK>', 'should', 'it', 'violence', '<s>', 'test', 'the', 'granted', 'there', 'offence', 'is', 'he', '<s>', ',', 'noticed', 'considerably', '<UNK>', 'in', '.', '.', 'criminal', '‘', 'provocation', 'in', ',', 'who', 'variety', 'juan', 'for', '.', 'the', 'force', 'still', 'bad', 'in', 'care', 'of', '—', 'driving', 'much', 'the', '<UNK>', 'citizen', 'on', '<UNK>', 'not', 'on', 'as', 'if', 'with', 'of', '</s>']
Trigram: ['<s>', 'they', 'all', 'went']


## Other information

In [16]:
def model_generation_time():
    generation_start = datetime.now()
    
    vanilla_uni(train_words)
    vanilla_bi(train_words)
    vanilla_tri(train_words)
    
    laplace_uni(train_words)
    laplace_bi(train_words)
    laplace_tri(train_words)
    
    unk_uni(train_words)
    unk_bi(train_words)
    unk_tri(train_words)
    
    generation_end = datetime.now()

    generation_time = dict()
    generation_time['generation_time'] = generation_end - generation_start
    print('Generation Time(HH::MM:SS:ms) - {}\n\n'.format(generation_time['generation_time']))

In [17]:
def RAMusage():
    pid = os.getpid()
    py = psutil.Process(pid)
    memoryUse = py.memory_info()[0]/2.**30
    print('Memory Use: ', memoryUse, 'GB')

In [18]:
print("Total size of corpus: "+str(len(tokens))+" words")
model_generation_time()
RAMusage()

Total size of corpus: 76004 words
Generation Time(HH::MM:SS:ms) - 0:00:00.387086


Memory Use:  0.14656829833984375 GB


# References

J. Daniel and J. Martin, “Speech and Language Processing,.” [Online]. Available: http://web.stanford.edu/~jurafsky/slp3/3.pdf.
‌